In [1]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 29.5MB/s 
     |████████████████████████████████| 460kB 31.2MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-05-31 07:43:21--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::22c3:9b0a, ...
Co

In [23]:
import numpy as np
import pandas as pd
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import files
myfile = files.upload()

Saving restaurant_review.txt to restaurant_review.txt


In [6]:
data_file = "./restaurant_review.txt"

# 열 제목과 레이블 지정
data = pd.read_table(data_file, names=['ratings', 'reviews'])
data['label'] = np.select([data.ratings>3], [1], default=0)

In [57]:
# 중복 리뷰 제거
data.drop_duplicates(subset=['reviews'], inplace=True)
# null 리뷰 제거
data = data.dropna()

# 한글과 공백을 제외한 나머지 문자 제거
data['reviews'] = data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data['reviews'].replace('', np.nan, inplace=True)
data = data.dropna()


In [158]:
# 훈련 데이터와 테스트 데이터 분리 (클래스 불균형 문제 해결)
positive_random = data[data['label']==1]
negative_random = data[data['label']==0]

train_data = pd.concat([positive_random[:300], negative_random[:300]])
test_data = pd.concat([positive_random[300:500], negative_random[300:500]])

# row 전체 shuffle
train_data = train_data.sample(frac=1)  
test_data = test_data.sample(frac=1)

In [133]:
# train_data = data[:4000]
# test_data = data[4000:]

In [159]:
# 토큰화 및 불용어 제거
# 불용어 정의
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
mecab = Mecab()

x_train = train_data['reviews'].apply(mecab.morphs)
x_train = x_train.apply(lambda x:[item for item in x if item not in stopwords])
y_train = train_data['label'].values

x_test = test_data['reviews'].apply(mecab.morphs)
x_test = x_test.apply(lambda x:[item for item in x if item not in stopwords])
y_test = test_data['label'].values

In [148]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(x_train)


# # 등장 횟수가 1인 단어들 배제
# threshold = 2
# # 단어의 수
# total_cnt = len(tokenizer.word_index)
# # 등장 빈도수가 threshold보다 작은 단어의 개수
# rare_cnt = 0
# # 훈련 데이터의 전체 단어 빈도수 총 합
# total_freq = 0
# # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수 총 합
# rare_freq = 0

# # 단어와 빈도수의 쌍
# for key, value in tokenizer.word_counts.items():
#     total_freq += value

#     # 단어의 등장 빈도수가 threshold 보다 작으면
#     if value < threshold:
#         rare_cnt += 1
#         rare_freq += value

# print("단어 집합의 크기: ", total_cnt)
# print('등장 빈도가 {} 이하인 희귀 단어 수 : {}'.format(threshold-1, rare_cnt))
# print("단어 집합에서 희귀 단어 비율 ; {}".format((rare_cnt/total_cnt)*100))
# print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {}'.format((rare_cnt/total_freq)*100))


# vocab_size = total_cnt-rare_cnt+2
# tokenizer = Tokenizer(vocab_size, oov_token='OOV')
# tokenizer.fit_on_texts(x_train)
# x_train = tokenizer.texts_to_sequences(x_train)
# x_test = tokenizer.texts_to_sequences(x_test)
# print(x_train[:3])

단어 집합의 크기:  4516
등장 빈도가 1 이하인 희귀 단어 수 : 2158
단어 집합에서 희귀 단어 비율 ; 47.78565101860053
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.9748761304820324
[[2, 13, 147, 19, 127, 83, 66], [259, 119, 48, 73, 40, 1, 22, 547, 233, 4, 133, 25, 49, 51, 1365, 1, 274, 835, 60, 22, 236, 1, 1714, 55, 476, 1, 1, 6, 270], [2, 3]]


In [161]:
#정수 인코딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
total_cnt = len(tokenizer.word_index)
vocab_size = total_cnt
tokenizer = Tokenizer(vocab_size, oov_token='OOV')
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [162]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if (len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s' % (max_len, (cnt / len(nested_list)) * 100))

max_len = 80
below_threshold_len(max_len, x_train)

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

전체 샘플 중 길이가 80 이하인 샘플의 비율: 97.66666666666667


In [163]:

from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)


Epoch 1/15
8/8 [==============================] - 3s 169ms/step - loss: 0.6900 - acc: 0.5146 - val_loss: 0.6867 - val_acc: 0.5417

Epoch 00001: val_acc improved from -inf to 0.54167, saving model to best_model.h5
Epoch 2/15
8/8 [==============================] - 1s 108ms/step - loss: 0.6775 - acc: 0.6417 - val_loss: 0.6710 - val_acc: 0.6167

Epoch 00002: val_acc improved from 0.54167 to 0.61667, saving model to best_model.h5
Epoch 3/15
8/8 [==============================] - 1s 111ms/step - loss: 0.6059 - acc: 0.7563 - val_loss: 0.6434 - val_acc: 0.6083

Epoch 00003: val_acc did not improve from 0.61667
Epoch 4/15
8/8 [==============================] - 1s 109ms/step - loss: 0.5117 - acc: 0.7604 - val_loss: 0.6140 - val_acc: 0.6417

Epoch 00004: val_acc improved from 0.61667 to 0.64167, saving model to best_model.h5
Epoch 5/15
8/8 [==============================] - 1s 111ms/step - loss: 0.4150 - acc: 0.8479 - val_loss: 0.5933 - val_acc: 0.6833

Epoch 00005: val_acc improved from 0.64167 

In [164]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(x_test, y_test)[1]))


13/13 [==============================] - 1s 17ms/step - loss: 0.9524 - acc: 0.5175

 테스트 정확도: 0.5175


In [165]:
def sentiment_predict(new_sentence):
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))


In [198]:
sentiment_predict('음식 진짜 맛있어요.. 완전 추천하는 식당입니다!')

96.51% 확률로 긍정 리뷰입니다.


In [188]:
sentiment_predict('음식에서 벌레 나왔어요.. 절대 안갈 식당입니다')

63.99% 확률로 부정 리뷰입니다.


In [189]:
sentiment_predict('그냥 맛없어요.. 메뉴를 다시 개발해보시는건 어떨지...')

97.71% 확률로 부정 리뷰입니다.


In [202]:
sentiment_predict('재방문 의사 완전 있어요')

66.61% 확률로 긍정 리뷰입니다.
